# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
A. Gould  ->  A. Gould  |  ['A. Gould']
J. Liu  ->  J. Liu  |  ['J. Liu']


C. Jäger  ->  C. Jäger  |  ['C. Jäger']
J. Lian  ->  J. Lian  |  ['J. Lian']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 73 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/
        2405.07152
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07367
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07514
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07559
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07855
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07964
      
Retrieving document from  https://arxiv.org/e-print/
        2405.07986
      


/tmp/ipykernel_2180/3009462391.py:49: LatexWarning: 
        2405.07152
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.07152' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2180/3009462391.py:49: LatexWarning: 
        2405.07367
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.07367' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2180/3009462391.py:49: LatexWarning: 
        2405.07514
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.07514' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2180/3009462391.py:49: LatexWarning: 
        2405.07559
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.07559' (found at least '\n')
  wa

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07152
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07152
      ) | **On the energy budget of starquake-induced repeating fast radio bursts**  |
|| W.-Y. Wang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-05-14*|
|*Comments*| *10 pages, 2 figures. Submitted. Some intriguing FAST's results are expected!*|
|**Abstract**|            With a growing sample of fast radio bursts (FRBs), we investigate the energy budget of different power sources within the framework of magnetar starquake triggering mechanism. During a starquake, the energy can be released in any form through magnetic, strain, rotational, and gravitational energies. Following findings are revealed: 1. The crust can store a free magnetic energy of the amount of at least $6.3\times10^{46}$ erg via toroidal fields, with frequent starquakes happening due to the instability of the crust. 2. The strain energy develops as a rigid object spins down, which can be released during a global starquake accompanied by a glitch. However, it takes a long time to accumulate enough strain energy via spin-down. 3. The rotational energy of a magnetar with $P\lesssim0.1\rm\,s$ can match the energy and luminosity budget of FRBs. 4. The budget of the total gravitational energy is high, but the mechanism and efficiency of converting this energy to radiation deserve further exploration.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07152' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07367
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07367
      ) | **TOI-2447 b / NGTS-29 b: a 69-day Saturn around a Solar analogue**  |
|| S. Gill, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Trifonov</mark> |
|*Appeared on*| *2024-05-14*|
|*Comments*| *16 pages, 12 figures. Accepted for publication in MNRAS*|
|**Abstract**|            Discovering transiting exoplanets with relatively long orbital periods ($>$10 days) is crucial to facilitate the study of cool exoplanet atmospheres ($T_{\rm eq} < 700 K$) and to understand exoplanet formation and inward migration further out than typical transiting exoplanets. In order to discover these longer period transiting exoplanets, long-term photometric and radial velocity campaigns are required. We report the discovery of TOI-2447 b ($=$ NGTS-29b), a Saturn-mass transiting exoplanet orbiting a bright (T=10.0) Solar-type star (T$_{\rm eff}$=5730 K). TOI-2447 b was identified as a transiting exoplanet candidate from a single transit event of 1.3% depth and 7.29 h duration in $TESS$ Sector 31 and a prior transit event from 2017 in NGTS data. Four further transit events were observed with NGTS photometry which revealed an orbital period of P=69.34 days. The transit events establish a radius for TOI-2447 b of $0.865 \pm 0.010\rm R_{\rm J}$, while radial velocity measurements give a mass of $0.386 \pm 0.025 \rm M_{\rm J}$. The equilibrium temperature of the planet is $414$ K, making it much cooler than the majority of $TESS$ planet discoveries. We also detect a transit signal in NGTS data not caused by TOI-2447 b, along with transit timing variations and evidence for a $\sim$150 day signal in radial velocity measurements. It is likely that the system hosts additional planets, but further photometry and radial velocity campaigns will be needed to determine their parameters with confidence. TOI-2447 b/NGTS-29b joins a small but growing population of cool giants that will provide crucial insights into giant planet composition and formation mechanisms.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07367' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07514
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07514
      ) | **KMT-2023-BLG-1866Lb: Microlensing super-Earth around an M dwarf host**  |
|| C. Han, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2024-05-14*|
|*Comments*| *9 pages, 8 figures, 4 tables*|
|**Abstract**|            We investigate the nature of the short-term anomaly that appears in the lensing light curve of KMT-2023-BLG-1866. The anomaly was only partly covered due to its short duration, less than a day, coupled with cloudy weather conditions and restricted nighttime duration. Considering intricacy of interpreting partially covered signals, we thoroughly explore all potential degenerate solutions. Through this process, we identify three planetary scenarios that equally well account for the observed anomaly. These scenarios are characterized by the specific planetary parameters: $(s, q)_{\rm inner} = [0.9740 \pm 0.0083, (2.46 \pm 1.07) \times 10^{-5}]$, $(s, q)_{\rm intermediate} = [0.9779 \pm 0.0017, (1.56 \pm 0.25)\times 10^{-5}]$, and $(s, q)_{\rm outer} = [0.9894 \pm 0.0107, (2.31 \pm 1.29)\times 10^{-5}]$, where $s$ and $q$ denote the projected separation (scaled to the Einstein radius) and mass ratio between the planet and its host, respectively. We identify that the ambiguity between the inner and outer solutions stems from the inner-outer degeneracy, while the similarity between the intermediate solution and the others is due to an accidental degeneracy caused by incomplete anomaly coverage. Through Bayesian analysis utilizing the constraints derived from measured lensing observables and blending flux, our estimation indicates that the lens system comprises a very low-mass planet orbiting an early M-type star situated approximately (6.2 -- 6.5)~kpc from Earth in terms of median posterior values for the different solutions. The median mass of the planet host is in the range of (0.48 -- 0.51)~$M_\odot$, and that of the planet's mass spans a range of (2.6 -- 4.0)~$M_{\rm E}$, varying across different solutions. The detection of KMT-2023-BLG-1866Lb signifies the extension of the lensing surveys to very low-mass planets that have been difficult to be detected from earlier surveys.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07514' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07559
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07559
      ) | **Preliminary Design of Detector Assembly for DIXE**  |
|| <mark>J. Liu</mark>, et al. |
|*Appeared on*| *2024-05-14*|
|*Comments*| *13 pages, 6 figures. Submitted version, the full version is published by Journal of Low Temperature Physics*|
|**Abstract**|            Diffuse X-ray Explorer (DIXE) is a proposed X-ray spectroscopic survey experiment for the China Space Station. Its detector assembly (DA) contains the transition edge sensor (TES) microcalorimeter and readout electronics based on the superconducting quantum interference device (SQUID) on the cold stage. The cold stage is thermally connected to the ADR stage, and a Kevlar suspension is used to stabilize and isolate it from the 4 K environment. TES and SQUID are both sensitive to the magnetic field, so a hybrid shielding structure consisting of an outer Cryoperm shield and an inner niobium shield is used to attenuate the magnetic field. In addition, IR/optical/UV photons can produce shot noise and thus degrade the energy resolution of the TES microcalorimeter. A blocking filter assembly is designed to minimize the effects. In it, five filters are mounted at different temperature stages, reducing the probability of IR/optical/UV photons reaching the detector through multiple reflections between filters and absorption. This paper will describe the preliminary design of the detector assembly and its optimization.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07559' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07855
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07855
      ) | **Formation of N-bearing complex organic molecules in molecular clouds: Ketenimine, acetonitrile, acetaldimine, and vinylamine via the UV photolysis of C$_2$H$_2$ ice**  |
|| K.-J. Chuang, <mark>C. Jäger</mark>, J. C. Santos, T. Henning |
|*Appeared on*| *2024-05-14*|
|*Comments*| *22 pages, 9 figures*|
|**Abstract**|            The solid-state C$_2$H$_2$ chemistry in interstellar H$_2$O-rich ice has been proposed to explain astronomically observed complex organic molecules (COMs), including ketene (CH$_2$CO), acetaldehyde (CH$_3$CHO), and ethanol (CH$_3$CH$_2$OH), toward early star-forming regions. This formation mechanism is supported by recent laboratory studies and theoretical calculations for the reactions of C$_2$H$_2$+OH/H. However, the analog reaction of C$_2$H$_2$+NH$_2$ forming N-bearing species has been suggested to have a relatively low rate constant that is orders of magnitude lower than the value of C$_2$H$_2$+OH. This work extends our previous laboratory studies on O-bearing COM formation to investigate the interactions between C$_2$H$_2$ and NH$_3$ ice triggered by cosmic ray-induced secondary UV photons under molecular cloud conditions. Experiments were performed in an ultra-high vacuum chamber to investigate the UV photolysis of the C$_2$H$_2$:NH$_3$ ice mixture at 10 K. The studied ice chemistry of C$_2$H$_2$ with NH$_2$ radicals and H atoms resulting from the UV photodissociation of NH$_3$ leads to the formation of several N-bearing COMs, including vinylamine (CH$_2$CHNH$_2$), acetaldimine (CH$_3$CHNH), acetonitrile (CH$_3$CN), ketenimine (CH$_2$CNH), and tentatively ethylamine (CH$_3$CH$_2$NH$_2$). The experimental results show an immediate and abundant CH$_2$CHNH$_2$ yield as the first-generation product, which is further converted into other chemical derivatives. The effective destruction and formation cross-section values of parent species and COMs were derived, and we discuss the chemical links among these molecules and their astronomical relevance.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07855' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07964
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07964
      ) | **Early phase simultaneous multi-band observations of Type II supernova SN 2024ggi with Mephisto**  |
|| X. Chen, et al. -- incl., <mark>J. Lian</mark> |
|*Appeared on*| *2024-05-14*|
|*Comments*| *Pages 9, Table 1, Figures 7*|
|**Abstract**|            We present early-phase good cadence simultaneous multi-band ($ugi$, $vrz$--bands) imaging of nearby supernova SN 2024ggi, which exploded in the nearby galaxy, NGC~3621. A quick follow-up was conducted within less than a day after the explosion and continued $\sim$23 days. The $uvg$-band light curves display a rapid rise ($\sim$1.4 mag day$^{-1}$) to maximum in $\sim$4 days and absolute magnitude $M_{g}\sim$--17.75 mag. The post-peak decay rate in redder bands is $\sim$0.01 mag day$^{-1}$. Different colors (e.g., $u-g$ and $v-r$) of SN~2024ggi are slightly redder than SN~2023ixf. A significant rise ($\sim$12.5 kK) in black-body temperature (optical) was noticed within $\sim$2 days after the explosion, which successively decreased, indicating shock break out inside a dense circumstellar medium (CSM) surrounding the progenitor. Using semi-analytical modeling, the ejecta mass and progenitor radius were estimated as 1.2 M$_{\odot}$ and $\sim$550 R$_{\odot}$, respectively. The archival deep images ($g,r,i,z$-bands) from the Dark Energy Camera Legacy Survey (DECaLS) were examined, and a possible progenitor was detected in each band ($\sim$22--22.5 mag) and had a mass range of 14--17 M$_{\odot}$.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07964' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.07986
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.07986
      ) | **JWST's PEARLS: resolved study of the stellar and dust components in starburst galaxies at cosmic noon**  |
|| M. Polletta, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-05-14*|
|*Comments*| *24 pages, 21 figures + appendix. Submitted to A&A. Comments welcome!*|
|**Abstract**|            Dusty star-forming galaxies (DSFGs) contribute significantly to the stellar buildup at cosmic noon. Major mergers and gas accretion are often invoked to explain DSFGs' prodigious star-formation rates (SFRs) and large stellar masses. We conducted a spatially-resolved morphological analysis of the rest-frame UV/NIR emission in three DSFGs at z~2.5. Initially discovered as CO emitters by NOEMA observations of a bright Herschel source, we observed them with the JWST/NIRCam as part of the PEARLS program. The NIRCam data reveal the galaxies' stellar population and dust distribution on scales of 250 pc. Spatial variations in stellar mass, SFR, and dust extinction are determined in resolved maps obtained through pixel-based SED fitting. The CO emitters are massive, dusty starburst galaxies with SFRs ranging from 340 to 2500 Msun/yr, positioning them among the most active SFGs at 2<z<3. Notably, they belong to the ~1.5% of the entire JWST population with extremely red colors. Their morphologies are disk-like, with effective radii of 2.0-4.4 kpc, and exhibit sub-structures such as clumps and spiral arms. The galaxies have dust extinctions up to Av=5-7 mag with asymmetric distributions extending over several kpc and including off-center regions resembling bent spiral arms and clumps. The NIR dust-attenuation curve in these sources deviates from standard laws, implying different dust grain properties than commonly assumed in starburst galaxies. The proximity of galaxies with consistent redshifts, strong color gradients, overall disturbed appearance, asymmetric dust obscuration, and wide-spread star formation favor interactions (minor mergers and flybys) as the mechanism driving the CO galaxies' exceptional SFRs. Their large masses and rich environment hint at membership in two proto-structures, as initially inferred from their association with a Planck-selected high-z source.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.07986' (found at least '\n')</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

414  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
